### Comparison between the A/E method and the hybrid approach autoencoder+A/E+classifier
The hybrid approach consists of training the classifier NN with both deep encoded and A/E features.

In [1]:
from begepro.autoencoder.sn_analysis import Comparison
from begepro.autoencoder.dataloading import DataLoader, Dataset
import numpy as np
import matplotlib.pyplot as plt
from begepro.autoencoder.autoencoder_classifier import Autoencoder, Classifier


2023-02-10 10:31:17.999574: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-10 10:31:18.363129: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-02-10 10:31:18.363148: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-02-10 10:31:19.391198: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [2]:
opts = {"loadpath": "/mnt/e/BEGE_data/waveforms_crioconite", # where .bin files are stored
"savepath": "../../dataset/Th232", # where .npy files will be saved with readfiles() and loaded with load()
"subsampling_factor": 3}

dloader = DataLoader(opts)
waveforms, _, energies, _, _ = dloader.load()

opts_2 = {"loadpath": "/mnt/e/BEGE_data/waveforms_crioconite", # where .bin files are stored
"savepath": "../../dataset/Th232_2", # where .npy files will be saved with readfiles() and loaded with load()
"subsampling_factor": 3}

dloader_2 = DataLoader(opts_2)
waveforms_2, _, energies_2, _, _ = dloader_2.load()

In [3]:
preds = np.load("../scripts/preds_hybrid.npy")[:,0]
ratio = np.load("../scripts/ratio.npy")
ratio_2 = np.load("../scripts/ratio_2.npy")

waveforms = np.concatenate([waveforms, waveforms_2])
ratio = np.concatenate([ratio, ratio_2])
energies = np.concatenate([energies, energies_2])

In [7]:
comparison = Comparison(energies, ratio, preds)
M_ELECTRON = 511
cs = np.linspace(0.0, 0.7, 30)
peaks = [2615, 2615 - 1*M_ELECTRON, 2615 - 2*M_ELECTRON, 1588, 1622, 1460, 860, 835, 409, 583, 2177]

In [8]:
sb_nn, std_sb_nn, sb_avse, std_sb_avse = comparison.compare(peaks, cs, benchmark = "nn")

Estimating S/B for peak at 2615 keV
Estimating S/B for peak at 2104 keV
Estimating S/B for peak at 1593 keV
Estimating S/B for peak at 1588 keV
Estimating S/B for peak at 1622 keV
Estimating S/B for peak at 1460 keV
Estimating S/B for peak at 860 keV
Estimating S/B for peak at 835 keV
Estimating S/B for peak at 409 keV
Estimating S/B for peak at 583 keV
Estimating S/B for peak at 2177 keV


In [9]:
for n in range(len(peaks)):
    plt.plot(cs, sb_nn[:,n])
    plt.fill_between(cs, sb_nn[:,n] + std_sb_nn[:,n], sb_nn[:,n] - std_sb_nn[:,n],label='_nolegend_', alpha = 0.5)
    plt.plot(cs, sb_avse[:,n])
    plt.fill_between(cs, sb_avse[:,n] + std_sb_avse[:,n], sb_avse[:,n] - std_sb_avse[:,n],label='_nolegend_', alpha = 0.5)
    plt.xlabel("NN score threshold")
    plt.ylabel("S/B ratio")
    plt.title(f"Peak at {peaks[n]}keV")
    plt.legend(["NN", "A/E"])
    plt.savefig(f"../../results/z_peak_at_{peaks[n]}_122_sampling_hybrid.svg")
    plt.clf()

<Figure size 640x480 with 0 Axes>